In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np

In [3]:
data_training = pd.read_csv ('dataTrain_carListings.csv')
data_testing = pd.read_csv ('dataTest_carListings.csv', index_col=0)

In [4]:
data_training.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [5]:
data_testing.head()

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand


In [6]:
data_training = data_training.drop(['Make'],axis=1)
data_testing = data_testing.drop(['Make'],axis=1)

In [7]:
data_testing.head()

,Year,Mileage,State,Model
ID,,,,
0,2014,31909,MD,MuranoAWD
1,2017,5362,FL,Wrangler
2,2014,50300,OH,FlexLimited
3,2004,132160,WA,5
4,2015,25226,MA,Grand


In [8]:
state_factorized = pd.factorize(data_training.State)
model_factorized = pd.factorize(data_training.Model)

data_training['State'] = state_factorized[0]
data_training['Model'] = model_factorized[0]

data_testing['State'] = pd.factorize(data_testing.State)[0]
data_testing['Model'] = pd.factorize(data_testing.Model)[0]

In [9]:
state_factorized, model_factorized

((array([ 0,  1,  2, ..., 13, 33,  3], dtype=int64),
  Index([' FL', ' OH', ' TX', ' CO', ' ME', ' WA', ' CT', ' CA', ' LA', ' NY',
         ' PA', ' SC', ' ND', ' NC', ' GA', ' AZ', ' TN', ' KY', ' NJ', ' UT',
         ' IA', ' AL', ' NE', ' IL', ' OK', ' MD', ' NV', ' WV', ' MI', ' VA',
         ' WI', ' MA', ' OR', ' IN', ' NM', ' MO', ' HI', ' KS', ' AR', ' MN',
         ' MS', ' MT', ' AK', ' VT', ' SD', ' NH', ' DE', ' ID', ' RI', ' WY',
         ' DC'],
        dtype='object')),
 (array([  0,   1,   2, ..., 118,  11, 237], dtype=int64),
  Index(['Wrangler', 'Tahoe4WD', 'X5AWD', 'SRXLuxury', '3', 'C-ClassC300',
         'CamryL', 'TacomaPreRunner', 'LaCrosse4dr', 'ChargerSXT',
         ...
         'WRXLimited', 'AccordSE', 'QX562WD', 'Escalade2WD', 'EscapeLImited',
         'PriusOne', 'Element2WD', 'Excursion137"', 'WRXPremium', 'RX-84dr'],
        dtype='object', length=525)))

In [10]:
list(model_factorized[1]).index('Tahoe4WD')

1

In [11]:
data_training.head()

,Price,Year,Mileage,State,Model
0,34995,2017,9913,0,0
1,37895,2015,20578,1,1
2,18430,2012,83716,2,2
3,24681,2014,28729,1,3
4,26998,2013,64032,3,0


In [12]:
X_training = data_training[['Year','Mileage','State','Model']]
y_training = data_training[['Price']]

X_training.head()
y_training.head()

,Price
0,34995
1,37895
2,18430
3,24681
4,26998


In [13]:
X_testing = data_testing
X_testing.head()


,Year,Mileage,State,Model
ID,,,,
0,2014,31909,0,0
1,2017,5362,1,1
2,2014,50300,2,2
3,2004,132160,3,3
4,2015,25226,4,4


In [14]:
'''
Este codigo busca la mejor combinacion de hiperparemetros, toma algunos minustos.

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Definir la cuadrícula de parámetros
param_grid = {
    'gamma': [0, 0.1, 0.5],
    'learning_rate': [0.01, 0.1, 0.5],
    'colsample_bytree': [0.5, 0.7, 1],
    'n_estimators': [100, 200, 300],
}

# Se aplica la estrategia GridSearch
grid_search = GridSearchCV(estimator=XGBRegressor(), param_grid=param_grid, cv=6, scoring='neg_mean_squared_error')
grid_search.fit(X_training, y_training)

# Obtener los mejores parámetros encontrados
print("Mejores parametros encontrados: ", grid_search.best_params_)
print("RMSE más bajo encontrado: ", np.sqrt(np.abs(grid_search.best_score_)))
'''

'\nEste codigo busca la mejor combinacion de hiperparemetros, toma algunos minustos.\n\nfrom xgboost import XGBRegressor\nfrom sklearn.model_selection import GridSearchCV\n\n# Definir la cuadrícula de parámetros\nparam_grid = {\n    \'gamma\': [0, 0.1, 0.5],\n    \'learning_rate\': [0.01, 0.1, 0.5],\n    \'colsample_bytree\': [0.5, 0.7, 1],\n    \'n_estimators\': [100, 200, 300],\n}\n\n# Se aplica la estrategia GridSearch\ngrid_search = GridSearchCV(estimator=XGBRegressor(), param_grid=param_grid, cv=6, scoring=\'neg_mean_squared_error\')\ngrid_search.fit(X_training, y_training)\n\n# Obtener los mejores parámetros encontrados\nprint("Mejores parametros encontrados: ", grid_search.best_params_)\nprint("RMSE más bajo encontrado: ", np.sqrt(np.abs(grid_search.best_score_)))\n'

In [15]:
from xgboost import XGBRegressor

xgb_reg_calibrado = XGBRegressor(gamma=0, learning_rate=0.5, colsample_bytree=1, n_estimators=300)

xgb_reg_calibrado.fit(X_training, y_training)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.5, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [16]:
y_pred = pd.DataFrame(xgb_reg_calibrado.predict(X_testing), index=X_testing.index, columns=['Price'])
# y_pred.to_csv('prediction.csv', index_label='ID')
y_pred

,Price
ID,
0,31994.548828
1,49987.886719
2,25524.439453
3,9355.213867
4,29221.847656
...,...
99995,22341.382812
99996,31235.216797
99997,30991.939453


In [17]:
year = 2014
miliage = 31909
state = list(state_factorized[1]).index(' FL')
model = list(model_factorized[1]).index('Wrangler')

sample = pd.DataFrame(
    {
        "Year":[year],
        "Mileage":[miliage],
        "State":[state],
        "Model":[model]
    }
)

x = xgb_reg_calibrado.predict(sample)
x[0]

31994.549

In [18]:
# Export the model

import joblib

joblib.dump(xgb_reg_calibrado, 'predictor_precios_model.pkl', compress=3)

['predictor_precios_model.pkl']

In [19]:
modelo = joblib.load('predictor_precios_model.pkl')

price = modelo.predict(sample)

print(price)

[31994.549]
